<img src="image.jpg">

 # PyPRT - Building Modeling Optimization

This notebook is used to show how the Python bindings for PRT can be employed in optimizing the rule attributes of a generated model to achive predefined performance goals.

### Import PyPRT module and define paths-related functions

In [1]:
import sys, os

sys.path.append(os.path.join(os.getcwd(), "..", "src"))
from utility import visualize_PRT_results, combine_reports

SDK_PATH = os.path.join(os.getcwd(), "..", "install", "bin")
sys.path.append(SDK_PATH)

import pyprt

In [2]:
from scipy import optimize

In [3]:
CS_FOLDER = os.getcwd()
def asset_file(filename):
    return os.path.join(CS_FOLDER, filename)

In [4]:
print("\nInitializing PRT.")
pyprt.initialize_prt(SDK_PATH)

if not pyprt.is_prt_initialized():
    raise Exception("PRT is not initialized")


Initializing PRT.
[PRT] [2019-08-23 13:26:37] [info] Esri Procedural Runtime
[PRT] [2019-08-23 13:26:37] [info] Version: 2.0.5403 | Build Date: Tue 04/09/2019 10:26 AM | Build Config: PRT_BC_REL PRT_CC_OPT PRT_TC_VC141 | OS: win32 | ARCH: x86_64
[PRT] [2019-08-23 13:26:37] [debug] prtPlugins item 0: 'C:\Users\cami9495\Documents\esri-cityengine-sdk-master\examples\py4prt\Demo2_Optimization\..\install\lib'
[PRT] [2019-08-23 13:26:37] [error] failed to load library: error while loading library 'C:\Users\cami9495\Documents\esri-cityengine-sdk-master\examples\py4prt\Demo2_Optimization\..\install\lib\com.esri.prt.unreal.dll': The specified module could not be found.

[PRT] [2019-08-23 13:26:37] [warning] failed to load library com.esri.prt.unreal.dll
[PRT] [2019-08-23 13:26:37] [error] failed to load library: error while loading library 'C:\Users\cami9495\Documents\esri-cityengine-sdk-master\examples\py4prt\Demo2_Optimization\..\install\lib\DatasmithSDK.dll': The specified module could not 

## Optimization Problem Definition

In this notebook, we are using a CGA rule that allows us to generate modern and sustainable buildings. The architecture of these buildings is focused on the green aspect of the building. Therefore, in addition to be modern and esthetical, the generated building answers also to the actual requirements of the city which are to encourage the integration of greenery on the buildings.

<img src="green_building.png">

### The variables

Among the values reported by the CGA rule, we want to optimize the __height of the generated building__. In this example, we would like the building height to be as close as possible to __10 meters__.
The next step is to define the inputs that will vary and on which we will optimize. These inputs must be CGA rule attributes. Here we will take the building height and the k factor as optimization inputs.

In [13]:
goal_str = "Building Height" + "_sum" # correspond to a CGA reported value

input_attr = ["buildingHeight","k"]
x0 = [float(13), float(0.1)] # input attributes initial value

In [5]:
shape_geo = asset_file("parcelN.obj") # parcel/geometry on which the model will be generated
mod = pyprt.ModelGenerator(shape_geo)

In [15]:
rpk = asset_file("simple_rule2.rpk")
attrs = {'ruleFile' : "bin/simple_rule2.cgb", 'startRule' : "Default$Footprint"}

model1 = mod.generate_model(rpk, attrs, {})

[PRT] [2019-08-22 13:44:06] [info] Using rule package C:\Users\cami9495\Documents\esri-cityengine-sdk-master\examples\py4prt\Demo2_Optimization\simple_rule2.rpk

[PRT] [2019-08-22 13:44:06] [debug] resolve map = <ResolveMap>
	<attribute key="bin/simple_rule2.cgb" value="rpk:file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk-master%5Cexamples%5Cpy4prt%5CDemo2_Optimization%5Csimple_rule2.rpk!/bin/simple_rule2.cgb" type="str"/>
	<attribute key="rules/simple_rule2.cga" value="rpk:file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk-master%5Cexamples%5Cpy4prt%5CDemo2_Optimization%5Csimple_rule2.rpk!/rules/simple_rule2.cga" type="str"/>
	<attribute key="/test-prt4cmd/rules/simple_rule2.cga" value="rpk:file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk-master%5Cexamples%5Cpy4prt%5CDemo2_Optimization%5Csimple_rule2.rpk!/rules/simple_rule2.cga" type="str"/>
</ResolveMap>

[PRT] [2019-08-22 13:44:06] [debug] trying to read initial shape geometry from file:/C:%5CUse

In [16]:
def fct(x): # function to minimize

    generated_mod = mod.generate_another_model({input_attr[0] : x[0] , input_attr[1] : x[1]}, {'emitGeometry' : False})
    
    if generated_mod:
        rep = combine_reports(generated_mod)[0]
        #return rep[goal_str] # output to optimize : y
        return ((-10+rep[goal_str])*(-10+rep[goal_str]))

    else:
        print("Error in optimization process.")  

## Optimization Problem Resolution

In [17]:
bounds = [(0,25),(0,0.6)] # sequence of (min,max)

In [18]:
res = optimize.differential_evolution(fct, bounds)
res

     fun: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 1953
     nit: 64
 success: True
       x: array([10.        ,  0.45008009])

The set of attributes that minimizes the output is: __building height of 10 meters__ and __factor k of 0.45__.

### Conclusion and Visualization

In [19]:
generated_model = mod.generate_another_model({input_attr[0] : res.x[0] , input_attr[1] : res.x[1]}, {'emitGeometry' : False})

if generated_model:
    visualize_PRT_results(generated_model)


Number of generated geometries (= nber of initial shapes):
1

Initial Shape Index: 0

Report of the generated model:
{'Building Height_avg': 10.0, 'Building Height_max': 10.0, 'Building Height_min': 10.0, 'Building Height_n': 1.0, 'Building Height_sum': 10.0, 'Id_avg': 0.0, 'Id_max': 0.0, 'Id_min': 0.0, 'Id_n': 1.0, 'Id_sum': 0.0, 'Parcel Area_avg': 1780.0284839748856, 'Parcel Area_max': 1780.0284839748856, 'Parcel Area_min': 1780.0284839748856, 'Parcel Area_n': 1.0, 'Parcel Area_sum': 1780.0284839748856, 'Value_avg': 1.7524919973620248, 'Value_max': 1.7524919973620248, 'Value_min': 1.7524919973620248, 'Value_n': 1.0, 'Value_sum': 1.7524919973620248}



In [20]:
mod = pyprt.ModelGenerator(shape_geo)
exported_model = mod.generate_model(rpk, attrs, {}, "com.esri.prt.codecs.OBJEncoder")

[PRT] [2019-08-09 13:40:44] [info] Using rule package C:\Users\cami9495\Documents\esri-cityengine-sdk-master\examples\py4prt\Demo2_Optimization\simple_rule2.rpk

[PRT] [2019-08-09 13:40:44] [debug] resolve map = <ResolveMap>
	<attribute key="bin/simple_rule2.cgb" value="rpk:file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk-master%5Cexamples%5Cpy4prt%5CDemo2_Optimization%5Csimple_rule2.rpk!/bin/simple_rule2.cgb" type="str"/>
	<attribute key="rules/simple_rule2.cga" value="rpk:file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk-master%5Cexamples%5Cpy4prt%5CDemo2_Optimization%5Csimple_rule2.rpk!/rules/simple_rule2.cga" type="str"/>
	<attribute key="/test-prt4cmd/rules/simple_rule2.cga" value="rpk:file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk-master%5Cexamples%5Cpy4prt%5CDemo2_Optimization%5Csimple_rule2.rpk!/rules/simple_rule2.cga" type="str"/>
</ResolveMap>

[PRT] [2019-08-09 13:40:44] [debug] trying to read initial shape geometry from file:/C:%5CUse

In [13]:
from PyGEL3D import gel
from PyGEL3D import js

os.chdir("../output")
js.set_export_mode()
m = gel.obj_load("theModel_0.obj")

ModuleNotFoundError: No module named 'PyGEL3D'

Visualization in a separate window:

In [19]:
viewer = gel.GLManifoldViewer()
viewer.display(m) # press ESC to go back to the script

In [12]:
viewer.event_loop()

In [20]:
del viewer

Visualization in a notebook cell:

In [21]:
js.display(m, smooth=False, wireframe=False)

Save the glTF model on ArcGIS Online (possible to download it from there)

In [21]:
exported_GLTF= mod.generate_model(rpk, attrs, {}, "com.esri.prt.codecs.GLTFEncoder")

[PRT] [2019-08-09 13:41:26] [debug] trying to read initial shape geometry from file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk-master%5Cexamples%5Cpy4prt%5CDemo2_Optimization%5CparcelN.obj


In [4]:
GLTF_item = gis.content.add({'title':'Test','type':"Service Definition"}, data='theModel_0.glb', folder='packages')

In [ ]:
display(GLTF_item)

### Optimization with envelope.rpk

In [6]:
goal_str_green = "green area" + "_sum" # correspond to a CGA reported value

input_attr_green = ["lot_coverage_parameter"]
x0_green = [float(10)]

In [7]:
rpk_green = asset_file("envelope1806.rpk")
attrs_green = {'ruleFile' : "rules/typology/envelope.cgb", 'startRule' : "Default$Lot"}

mod2 = pyprt.ModelGenerator(shape_geo)
model = mod2.generate_model(rpk_green, attrs_green, {})

[PRT] [2019-08-23 13:26:54] [info] using rule package C:\Users\cami9495\Documents\esri-cityengine-sdk-master\examples\py4prt\Demo2_Optimization\envelope1806.rpk

[PRT] [2019-08-23 13:26:54] [debug] resolve map = <ResolveMap>
	<attribute key="/ESRI.lib/rules/Plants/Plant_Loader.cga" value="rpk:file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk-master%5Cexamples%5Cpy4prt%5CDemo2_Optimization%5Cenvelope1806.rpk!/.ws/ESRI.lib/rules/Plants/Plant_Loader.cga" type="str"/>
	<attribute key="/green_optimization_2/assets/vertical_greenery/6.PNG" value="rpk:file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk-master%5Cexamples%5Cpy4prt%5CDemo2_Optimization%5Cenvelope1806.rpk!/assets/vertical_greenery/6.PNG" type="str"/>
	<attribute key="/green_optimization_2/assets/vertical_greenery/8.PNG" value="rpk:file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk-master%5Cexamples%5Cpy4prt%5CDemo2_Optimization%5Cenvelope1806.rpk!/assets/vertical_greenery/8.PNG" type="str"/>
	<att

In [8]:
def fct_green(x):

    generated_mod_green = mod2.generate_another_model({input_attr_green[0] : x[0]} , {'emitGeometry' : False})
    
    if generated_mod_green:
        rep_green = combine_reports(generated_mod_green)[0]
        return -rep_green[goal_str_green]
    else:
        print("Error in optimization process.")

In [9]:
bounds_green = [(0,100)]

In [10]:
fct_green(x0_green)

-1239.5150093941356

In [11]:
import time
start = time.time()
res_green = optimize.differential_evolution(fct_green, bounds_green) # shgo
duration = time.time() - start
print("The optimization lasted: " + str(duration))
res_green

The optimization lasted: 36.950034618377686


     fun: -1625.8093735264208
 message: 'Optimization terminated successfully.'
    nfev: 62
     nit: 3
 success: True
       x: array([37.59567648])

In [12]:
attrs_green[input_attr_green[0]] = res_green.x[0]
green_model = mod2.generate_another_model(attrs_green)

if green_model:
    visualize_PRT_results(green_model)


Number of generated geometries (= nber of initial shapes):
1

Initial Shape Index: 0

Report of the generated model:
{'green area_avg': 4.1129430982990804, 'green area_max': 635.2412952931099, 'green area_min': 0.4894099778754496, 'green area_n': 395.0, 'green area_sum': 1624.6125238281368, 'total report for optimisation_avg': 4.1129430982990804, 'total report for optimisation_max': 635.2412952931099, 'total report for optimisation_min': 0.4894099778754496, 'total report for optimisation_n': 395.0, 'total report for optimisation_sum': 1624.6125238281368}



In [14]:
print("\nShutdown PRT.")
pyprt.shutdown_prt()


Shutdown PRT.
